In [49]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import SGD
from keras.utils import np_utils
from tensorflow.keras import Sequential, layers, losses, optimizers, models
import tensorflow as tf
import os
from csv import reader
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [50]:
seed_num = 128
import random as python_random
np.random.seed(seed_num)
python_random.seed(seed_num)
tf.random.set_seed(seed_num)

In [51]:
from pickle import load
load_minmax_scaler = load(open('./minmax_scaler.pkl', 'rb'))

In [52]:
checkpoint_path = './temp/checkpoint.ckpt'
optimizer = optimizers.Adam()
model = Sequential()
model.add(tf.keras.layers.Dense(38, input_dim =38,activation = 'relu'))
model.add(tf.keras.layers.Dense(19,activation = 'relu'))
model.add(tf.keras.layers.Dense(19,activation = 'relu'))
model.add(tf.keras.layers.Dense(9,activation = 'relu'))
model.add(tf.keras.layers.Dense(3,  activation='softmax'))
model.compile(
    loss=sparse_categorical_crossentropy, 
    optimizer=optimizer, 
    metrics=['acc'])
model.summary()
model.load_weights(checkpoint_path)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 38)                1482      
                                                                 
 dense_11 (Dense)            (None, 19)                741       
                                                                 
 dense_12 (Dense)            (None, 19)                380       
                                                                 
 dense_13 (Dense)            (None, 9)                 180       
                                                                 
 dense_14 (Dense)            (None, 3)                 30        
                                                                 
Total params: 2,813
Trainable params: 2,813
Non-trainable params: 0
_________________________________________________________________


In [53]:
excel_data=pd.read_csv('Total/ResultWeight_MainDimension.csv',header=None)

In [54]:
num_data = excel_data.to_numpy()
inx = [1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,18,20,22,24,25,26,27,28,29,30,31,32,33,34,35,36]
data1 = num_data[:,inx]

In [55]:
len(inx)

31

In [56]:
폭 = np.average(num_data[:,37:41],axis=1)
높이 = np.average(num_data[:,41:45],axis=1)
면적 = np.average(num_data[:,49:53],axis=1)
둘레 = np.average(num_data[:,53:57],axis=1)

면적둘레 = 면적/둘레
면적폭 = 면적/폭
면적높이 = 면적/높이
폭=np.reshape(폭,(num_data.shape[0],1))
높이=np.reshape(높이,(num_data.shape[0],1))
면적=np.reshape(면적,(num_data.shape[0],1))
둘레=np.reshape(둘레,(num_data.shape[0],1))
면적둘레=np.reshape(면적둘레,(num_data.shape[0],1))
면적폭=np.reshape(면적폭,(num_data.shape[0],1))
면적높이=np.reshape(면적높이,(num_data.shape[0],1))

In [57]:
data2 = np.concatenate((data1,폭,높이,면적,둘레,면적둘레,면적폭,면적높이,num_data[:,57:59]),axis=1)
data2[0]

array([860.01, 437.81, 351.94, 605.38, 174489.22, 1628.11, 342.24, 362.24,
       629.18, 655.27, 180697.82, 196069.74, 1628.76, 1674.88, 368.91,
       673.74, 207690.97, 1733.3, 371.8, 366.15, 657.88, 658.86,
       195073.62, 204146.25, 1707.21, 1713.75, 417.23, 375.93, 608.3,
       178965.78, 1612.7, 296.21000000000004, 494.5075, 150501.885,
       1287.5925, 116.8862703067935, 508.0918436244556, 304.3470220370773,
       269, 3], dtype=object)

In [58]:

num_data2 = data2

돼지파일명 = num_data[0:,0]
치수 = num_data2[0:,0:-2]
체형 = num_data2[0:,-1].astype('float32')
print(f'치수_shape = {치수.shape}\n체형_shape = {체형.shape}') #35개의 선별 치수 + 둘레, 폭, 높이당 면적

치수_shape = (798, 38)
체형_shape = (798,)


In [59]:
result = model.predict(load_minmax_scaler.transform(치수))
Results = np.argmax(result,axis =1)
Results = Results +1
#체형[696:] = Results[696:]
#num_data[697:,-1] = Results[696:]

25/25 [==============================] - 0s 831us/step


In [60]:
cnt = 0
for i in range(치수.shape[0]):
    if(Results[i] == 체형[i]):
        cnt = cnt +1
print(f"acc is {cnt/치수.shape[0]*100}")


acc is 91.72932330827066


In [61]:
index = []
cnt =0
for i,m in zip(range(치수.shape[0]),result.max(axis=1)):
    if(Results[i] == 체형[i]):
        cnt=cnt+1
        index.append(i)
    else:
        if(m < 0.7):
            index.append(i)
        
print(f'{cnt} of {len(index)} are correct [{cnt/len(index)*100}%]')

732 of 748 are correct [97.86096256684492%]


In [72]:
np.random.shuffle(index)

In [73]:
cnt =0

for i in range(200):
    if(Results[index][i] == 체형[index][i]):
        cnt = cnt +1
print(f"acc is {cnt/200*100}")

acc is 98.0


In [64]:
num_data[index][:100].shape

(100, 59)

In [65]:
merge = np.concatenate((num_data[index][:200],np.reshape(Results[index][:200],(200,1))),axis =1)

In [66]:
TF =[]
for i in range(200):
    if(merge[i][-1]==merge[i][-2]):
        TF.append("True")
    else:
        TF.append("False")

In [67]:
TF = np.array(TF)
TF = np.reshape(TF,(200,1))

### 측정 데이터 제거

In [68]:
merge = np.concatenate((merge[:,0:1],merge[:,-2:]),axis=1)
merge = np.concatenate((merge,TF),axis = 1)
merge

array([['5382_W295_T3_20210930_08_15_24_911_278.50kg_2.9type_Clipped_c1.pcd',
        3, 3, 'True'],
       ['6570_W289_T3_000000006570000_20210930_08_13_14_565_320.13kg_3.0type_Clipped_c1.pcd',
        3, 3, 'True'],
       ['7013_W186_T2_000000007013000_20210527_08_15_16_010.pcd', 2, 2,
        'True'],
       ['6247_W235_T2_000000006247000_20211007_08_12_36_366_308.76kg_3.4type_Clipped_c2.pcd',
        2, 2, 'True'],
       ['6674_W203_T1_20210930_08_09_39_482_187.44kg_1.5type_Clipped_c3.pcd',
        1, 1, 'True'],
       ['7678_W213_T3_000000007678000_20210930_08_15_17_665_159.24kg_2.0type_Clipped_c8.pcd',
        3, 3, 'True'],
       ['6818_W218_T2_000000006818000_20211007_08_07_40_089_253.33kg_1.5type_Clipped_c1.pcd',
        2, 2, 'True'],
       ['5382_W295_T3_20210930_08_15_24_977_314.15kg_2.7type_Clipped_c2.pcd',
        3, 3, 'True'],
       ['5604_W229_T2_000000195604000_20210527_08_40_21_166_0.00kg_0.0type_Clipped.pcd',
        2, 2, 'True'],
       ['6988_W168_T1_000000

### 엑셀 생성

In [69]:
돼지선별 = pd.DataFrame(merge)

In [71]:

돼지선별.to_excel('돼지_100.xlsx')